# Autograd：自动求导
torch.autograd是pytorch自动求导的工具，也是所有神经网络的核心。我们首先先简单了解一下这个包如何训练神经网络。

背景介绍

神经网络(NNs)是作用在输入数据上的一系列嵌套函数的集合，这些函数由权重和误差来定义，被存储在PyTorch中的tensors中。
神经网络训练的两个步骤：
前向传播：在前向传播中，神经网络通过将接收到的数据与每一层对应的权重和误差进行运算来对正确的输出做出最好的预测。
反向传播：在反向传播中，神经网络调整其参数使得其与输出误差成比例。反向传播基于梯度下降策略，是链式求导法则的一个应用，以目标的负梯度方向对参数进行调整。
更加详细的介绍可以参照下述地址：

[3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)

## 1. Pytorch应用

来看一个简单的示例，我们从torchvision加载一个预先训练好的resnet18模型，接着创建一个随机数据tensor来表示一有3个通道、高度和宽度为64的图像，其对应的标签初始化为一些随机值。

In [2]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)
print(data.shape,'--->',labels.shape)

torch.Size([1, 3, 64, 64]) ---> torch.Size([1, 1000])


接下来，我们将输入数据向输出方向传播到模型的每一层中来预测输出，这就是前向传播。

In [ ]:
prediction = model(data)# 前向传播

我们利用模型的预测输出和对应的权重来计算误差，然后反向传播误差。完成计算后，您可以调用.backward()并自动计算所有梯度。此张量的梯度将累积到.grad属性中。

In [ ]:
loss = (prediction - labels).sum()
loss.backward() #反向传播

接着，我们加载一个优化器，在本例中，SGD的学习率为0.01，momentum 为0.9。我们在优化器中注册模型的所有参数。

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0,9)

最后，我们调用.step()来执行梯度下降，优化器通过存储在.grad中的梯度来调整每个参数。

In [ ]:
optim.step()# 梯度下降

现在，你已经具备了训练神经网络所需所有条件。下面几节详细介绍了Autograd包的工作原理——可以跳过它们。

## 2.Autograd中的求导

先来看一下autograd是如何收集梯度的。我们创建两个张量a和b并设置requires_grad = True以跟踪它的计算。

In [6]:
import torch
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
print(a)
print(b)

tensor([2., 3.], requires_grad=True)
tensor([6., 4.], requires_grad=True)


接着在a和b的基础上创建张量Q
$Q=3 a^{3}-b^{2}$

In [7]:
Q = 3*a**3 - b**2
print(Q)

tensor([-12.,  65.], grad_fn=<SubBackward0>)


假设a和b是一个神经网络的权重，Q是它的误差，在神经网络训练中，我们需要w.r.t参数的误差梯度，即

$\frac{\partial Q}{\partial a}=9 a^{2}$
$\frac{\partial Q}{\partial b}=-2 b$
当我们调用Q的.backward()时，autograd计算这些梯度并把它们存储在张量的 .grad属性中。我们需要在Q.backward()中显式传递gradient，gradient是一个与Q相同形状的张量，它表示Q w.r.t本身的梯度，即
$\frac{d Q}{d Q}=1$
同样，我们也可以将Q聚合为一个标量并隐式向后调用，如Q.sum().backward()。

In [9]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

现在梯度都被存放在a.grad和b.grad中

In [11]:
print(9*a**2  == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])
